In [2]:
import os
from DeepBloomLib.DeepBloom import DeepBloom
from DeepBloomLib.DeeperBloom import DeeperBloom
from DeepBloomLib.Model import Model
from DeepBloomLib.AlmostPerfectModel import AlmostPerfectModel
from DeepBloomLib.PerfectModel import PerfectModel
from DeepBloomLib.GRUModel import GRUModel
from DeepBloomLib.AlwaysNoModel import AlwaysNoModel
from SandwichedLearnedBloom import SandwichedLearnedBloom
import json
import random
import string
import tensorflow as tf
from keras import backend as K
import pandas as pd
from DeepBloomLib.utils import * 

In [ ]:
with open('dataset.json', 'r') as f:
    dataset = json.load(f)

In [ ]:
positives = pd.Series(dataset["positives"])
positives = positives.append(pd.Series(dataset["negatives"]), ignore_index=True)
positives = pd.Series(positives.str.replace("www.","", regex=False))
positives =  pd.Series(positives.unique())
positives.to_csv("../Data/positive_urls.csv", header=['Domain'], index=False)

In [ ]:
positives = pd.read_csv("../Data/positive_urls.csv")
negatives = pd.read_csv("../Data/top10milliondomains.csv")
negatives = negatives[~negatives["Domain"].isin(positives["Domain"])]
negatives["Domain"] = negatives["Domain"].str.replace( "_", "", regex=False)
negatives.to_csv("../Data/top10milliondomains_CLEANED.csv", index=False)

In [3]:
negatives = pd.read_csv("../Data/top10milliondomains_CLEANED.csv")
negatives["RealPageRank"] = 10**negatives["Open Page Rank"]
negatives["PMF"] = negatives["RealPageRank"]/negatives["RealPageRank"].sum()
positives = pd.read_csv("../Data/positive_urls.csv")

In [6]:
train_dev_fraction = .8
deeper_bloom=False
fp_rate=0.01
fp_fractions=None

In [4]:
def generate_zipf_pmf(zipf_param, num_elements):
    pmf = []
    total = 0
    for i in range(1, num_elements+1):
        element = 1/(i**zipf_param)
        total += element
        pmf.append(element)
    for i in range(len(pmf)):
        pmf[i] /= total
    return pmf

In [ ]:
def train_model_and_save(path, positive_urls, negative_urls, number_of_known_negatives, batch_size_param=16384, epochs_param=100):
    ## Split negative data into subgroups. Saving some for choosing a threshold later.
    (s1, s2) = split_negatives(Data(positive_urls,negative_urls[:number_of_known_negatives]))
    print("Training model with train, dev, positives", len(s1), len(s2), len(positive_urls))
    ## Shuffle together subset of negatives and positives.
    ## Then, train the model on this data.
    shuffled = shuffle_for_training(s1, positive_urls)
    model = GRUModel('DeepBloomLib/glove.6B.50d-char.txt', 50, epochs=epochs_param, batch_size=batch_size_param)
    model.fit(shuffled[0], shuffled[1])
    print("Done fitting")
    model.save_model(path)

In [ ]:
train_model_and_save("../SavedModels/HALF_NEGATIVES_MODEL_2.h5", positives["Domain"], negatives["Domain"], 5000000, epochs_param=100)

In [ ]:
def train_deep_bloom_filter(positives, train_dev_negatives, model, fp_rate):
    with tf.device('/gpu:0'):
        db_1 = DeepBloom(model, Data(positives, train_dev_negatives), fp_rate)
        print("Params needed", db_1.model.model.count_params())
        return db_1

In [ ]:
def train_and_evaluate(bloom_filter_sizes, bloom_filter_fprs, target_fpr, positive_urls, negative_urls, number_of_known_negatives, pmf_weights):
    model = GRUModel('DeepBloomLib/glove.6B.50d-char.txt', 50, epochs=100, batch_size=16384)
    train_dev_negatives = negative_urls[:number_of_known_negatives]
    print("Number train, dev", len(train_dev_negatives))
    print("Number positives ", len(positive_urls))
    db = train_deep_bloom_filter(positive_urls, train_dev_negatives, model, target_fpr)
    model_false_positive_rate = 0.0
    print("Testing Negatives")
    model_false_positive_rate = float(np.sum((db.model.predicts(negative_urls) > db.threshold)*pmf_weights))
    print("Finished Testing Negatives")
    total_fpr = db.bloom_filter.fp_prob+model_false_positive_rate
    bloom_filter_sizes.append(db.bloom_filter.size)
    bloom_filter_fprs.append(total_fpr)
    print("Target false positive rate: ", target_fpr)
    print("Test false positive rate: ", total_fpr)
    print("Bloom filter bits needed", db.bloom_filter.size)

In [ ]:
def sweep_beta(min_number, max_number, steps,target_fpr, path, positive_urls, negative_urls):
    zipf_pmf = generate_zipf_pmf(.75, len(negatives))
    sizes = []
    fprs = []
    negatives_range = np.geomspace(min_number, max_number, steps).astype(np.int32)
    for i in range(steps):
        train_and_evaluate(sizes, fprs, target_fpr, positive_urls, negative_urls, negatives_range[i], zipf_pmf)
        saved_data = pd.DataFrame()
        saved_data["Total FPR"] = fprs
        saved_data["Size of Backup"] = sizes
        saved_data["Bits Per Element"] = (saved_data["Size of Backup"]+2353*4*8)/len(positive_urls)
        saved_data["Number of Negatives"] = negatives_range[:i+1]
        saved_data.to_csv(path, index=False)

In [ ]:
positives_subset = positives[:140000]
sweep_beta(1011342, 1400000, 2, .01,"../Data/url_learned_bloom_beta_sweep_09_29_restart.csv", positives_subset, negatives)

In [ ]:
positives_subset = positives[:140000]
sweep_beta(527763, 1400000, 4, .005,"../Data/url_learned_bloom_beta_sweep_09_29_005.csv", positives_subset, negatives)

In [ ]:
positives_subset = positives[:140000]
sweep_beta(1400000, 1400000, 8, .001,"../Data/url_learned_bloom_beta_sweep_09_29_001_1.csv", positives_subset, negatives)

In [ ]:
def sweep_zipf(min_zipf, max_zipf, steps, target_fpr, path, positive_urls, negative_urls, number_of_known_negatives):
    sizes = []
    fprs = []
    zipf_range = np.linspace(min_zipf, max_zipf, steps).astype(np.float32)
    model = GRUModel('DeepBloomLib/glove.6B.50d-char.txt', 50, epochs=100, batch_size=16384)
    train_dev_negatives = negative_urls[:number_of_known_negatives]
    print("Number train, dev", len(train_dev_negatives))
    print("Number positives ", len(positive_urls))
    db = train_deep_bloom_filter(positive_urls, train_dev_negatives, model, target_fpr)
    print("Testing Negatives")
    predictions = db.model.predicts(negative_urls) > db.threshold
    print("Finished Testing Negatives")
    for i in range(steps):
        model_false_positive_rate = float(np.sum(predictions*generate_zipf_pmf(zipf_range[i], len(negative_urls))))
        total_fpr = db.bloom_filter.fp_prob+model_false_positive_rate
        sizes.append(db.bloom_filter.size)
        fprs.append(total_fpr)
        saved_data = pd.DataFrame()
        saved_data["Total FPR"] = fprs
        saved_data["Size of Backup"] = sizes
        saved_data["Bits Per Element"] = (saved_data["Size of Backup"]+2353*4*8)/len(positive_urls)
        saved_data["Number of Negatives"] = number_of_known_negatives
        saved_data["Zipf"] = zipf_range[:i+1]
        saved_data.to_csv(path, index=False)
    print("Target false positive rate: ", target_fpr)
    print("Test false positive rate: ", total_fpr)
    print("Bloom filter bits needed", db.bloom_filter.size)

In [ ]:
sweep_zipf(0, 1.1, 12, .01, "../Data/url_learned_bloom_zipf_sweep_01.csv", positives[:140000], negatives, 280000)

In [ ]:
sweep_zipf(0, 1.2, 12, .005, "../Data/url_learned_bloom_zipf_sweep_005.csv", positives[:140000], negatives, 280000)

In [ ]:
sweep_zipf(0, 1.2, 12, .001, "../Data/url_learned_bloom_zipf_sweep_001.csv", positives[:140000], negatives, 280000)

In [ ]:
def sweep_data_size(min_number, max_number, steps, negatives_over_positives, universe_size_over_known, target_fpr, path, positive_urls, negative_urls):
    sizes = []
    fprs = []
    positives_range = np.linspace(min_number, max_number, steps).astype(np.int32)
    negatives_range = positives_range*negatives_over_positives
    negatives_universe_range = negatives_range*universe_size_over_known
    for i in range(steps):
        zipf_pmf = generate_zipf_pmf(.75, negatives_universe_range[i])
        train_and_evaluate(sizes, fprs, target_fpr, positive_urls[:positives_range[i]], negative_urls[:negatives_universe_range[i]], negatives_range[i], zipf_pmf)
        saved_data = pd.DataFrame()
        saved_data["Total FPR"] = fprs
        saved_data["Size of Backup"] = sizes
        saved_data["Bits Per Element"] = (saved_data["Size of Backup"]+2353*4*8)/len(positive_urls)
        saved_data["Number of Known Negatives"] = negatives_range[:i+1]
        saved_data["Size of Negative Universe"] = negatives_universe_range[:i+1]
        saved_data["Number of Positives"] = positives_range[:i+1]
        saved_data.to_csv(path, index=False)

In [ ]:
sweep_data_size(217142, 300000, 3, 2, 2, .01, "../Data/url_learned_bloom_size_sweep_01_RESTART.csv", positives, negatives)

In [ ]:
sweep_data_size(300000, 300000, 1, 2, 2, .005, "../Data/url_learned_bloom_size_sweep_005_RESTART.csv", positives, negatives)

In [ ]:
sweep_data_size(300000, 300000, 1, 2, 2, .001, "../Data/url_learned_bloom_size_sweep_001_RESTART.csv", positives, negatives)

In [ ]:
model = GRUModel('DeepBloomLib/glove.6B.50d-char.txt', 50, epochs=1, batch_size=16384)
model.load_model("../SavedModels/HALF_NEGATIVES_MODEL.h5", positives["Domain"], negatives["Domain"][:5000000])
print("Number train, dev", 5000000)
print("Number positives ", len(positives))
db = DeepBloom(model, Data(positives["Domain"], negatives["Domain"][:5000000]), .01, train_model=False)

In [ ]:
%%timeit
db.model.predicts(negatives["Domain"][:1000000])

In [ ]:
# 39.2 seconds to predict on 1,000,000 urls using the GPU
# 0.0000392 seconds per prediction

In [ ]:
model = GRUModel('DeepBloomLib/glove.6B.50d-char.txt', 50, epochs=1, batch_size=16384)
model.load_model("../SavedModels/HALF_NEGATIVES_MODEL.h5", positives["Domain"], negatives["Domain"][:5000000])
print("Number train, dev", 5000000)
print("Number positives ", len(positives))
db = DeepBloom(model, Data(positives["Domain"], negatives["Domain"][:5000000]), .01, train_model=False)

In [ ]:
%%timeit
db.model.predicts(negatives["Domain"][:1000000])

In [ ]:
# 

In [ ]:
def sweep_fpr(min_fpr, max_fpr, steps, pmf, path, positive_urls, negative_urls, number_of_known_negatives,epochs):
    sizes = []
    zipfs = []
    fprs = []
    fpr_range = np.geomspace(min_fpr, max_fpr, steps).astype(np.float32)
    model = GRUModel('DeepBloomLib/glove.6B.50d-char.txt', 50, epochs=epochs, batch_size=16384)
    model.load_model("../SavedModels/HALF_NEGATIVES_MODEL_2.h5", positive_urls, negative_urls[:number_of_known_negatives])
    print("Number train, dev", number_of_known_negatives)
    print("Number positives ", len(positive_urls))
    db = DeepBloom(model, Data(positive_urls, negative_urls[:number_of_known_negatives]), .01, train_model=False)
    print("Params needed", db.model.model.count_params())
    sample = pd.Series(np.random.choice(negative_urls, 1000000, True, pmf))
    for i in range(steps):
        db.fp_rate = fpr_range[i]
        db.create_fake_bloom_filter(Data(positive_urls, negative_urls[:number_of_known_negatives]))
        print("Calculating Predictions")
        predictions =  db.model.predicts(sample)
        print("Checking against threshold")
        predictions = [pred >db.threshold for pred in predictions]
        print("Calculating FPR")
        model_false_positive_rate = np.sum(predictions)/len(sample)
        total_fpr = db.bloom_filter.fp_prob+model_false_positive_rate
        sizes.append(db.bloom_filter.size)
        fprs.append(total_fpr)
        zipfs.append(.75)
        saved_data = pd.DataFrame()
        saved_data["Total FPR"] = fprs
        saved_data["Size of Backup"] = sizes
        saved_data["Bits Per Element"] = (saved_data["Size of Backup"]+2369*4*8)/len(positive_urls)
        saved_data["Number of Negatives"] = number_of_known_negatives
        saved_data["Zipf"] = zipfs
        saved_data.to_csv(path, index=False)
    print("Test false positive rate: ", total_fpr)
    print("Bloom filter bits needed", db.bloom_filter.size)

In [ ]:
sweep_fpr(.1, .001, 4, negatives["PMF"], "../Data/url_learned_bloom_fpr_sweep_HALF_DATA__3.csv", positives["Domain"], negatives["Domain"],  5000000, epochs=50)

In [ ]:
sweep_fpr(.01, .00005, 10, negatives["PMF"], "../Data/url_learned_bloom_fpr_sweep_QUARTER_DATA.csv", positives["Domain"], negatives["Domain"], 2500000, epochs=50)

In [ ]:
def sweep_learnability(min_swap, max_swap, steps, zipf, target_fpr, path, positive_urls, number_of_positives, negative_urls, number_of_known_negatives):
    sizes = []
    fprs = []
    swap_range = np.linspace(min_swap, max_swap, steps)
    train_dev_negatives = negative_urls[:number_of_known_negatives]
    print("Number train, dev", len(train_dev_negatives))
    print("Number positives ", len(positive_urls))
    for i in range(steps):
        num_positives_to_swap = int(swap_range[i]*number_of_positives)
        num_negatives_to_swap = int(swap_range[i]*number_of_known_negatives)
        mixed_positives = positive_urls[:number_of_positives]
        extra_positives = positive_urls[number_of_positives:]
        mixed_negatives = train_dev_negatives
        mixed_positives = mixed_negatives[:num_positives_to_swap] + mixed_positives[num_positives_to_swap:]
        mixed_negatives =  extra_positives[:num_negatives_to_swap] + mixed_negatives[num_negatives_to_swap:]
        test_negatives_set = mixed_negatives + negative_urls[len(negative_urls)-int((1-swap_range[i])*(1400000-len(mixed_negatives))):] + positive_urls[len(positive_urls)-int(swap_range[i]*((1400000-len(mixed_negatives)))):]
        np.random.shuffle(mixed_positives)
        np.random.shuffle(mixed_negatives)
        np.random.shuffle(test_negatives_set)
        model = GRUModel('DeepBloomLib/glove.6B.50d-char.txt', 50, epochs=100, batch_size=2048)
        db = train_deep_bloom_filter(mixed_positives, mixed_negatives, model, target_fpr)
        model_false_positive_rate = float(np.sum((db.model.predicts(test_negatives_set) > db.threshold)*(generate_zipf_pmf(zipf, len(test_negatives_set)))))
        total_fpr = db.bloom_filter.fp_prob+model_false_positive_rate
        sizes.append(db.bloom_filter.size)
        fprs.append(total_fpr)
        saved_data = pd.DataFrame()
        saved_data["Total FPR"] = fprs
        saved_data["Size of Backup"] = sizes
        saved_data["Bits Per Element"] = (saved_data["Size of Backup"]+2353*4*8)/number_of_positives
        saved_data["Number of Negatives"] = number_of_known_negatives
        saved_data["Zipf"] = zipf
        saved_data["Percent Positives Swapped"] = swap_range[:i+1]
        saved_data.to_csv(path, index=False)
    print("Target false positive rate: ", target_fpr)
    print("Test false positive rate: ", total_fpr)
    print("Bloom filter bits needed", db.bloom_filter.size)

In [ ]:
sweep_learnability(0, .1, 10, .75, .01, "../Data/url_learned_bloom_learnability_sweep_REDO.csv", positives, 140000, negatives, 560000)

In [ ]:
sweep_learnability(0, .1, 10, .75, .001, "../Data/url_learned_bloom_learnability_sweep_001.csv", positives, 140000, negatives, 560000)

In [ ]:
def sweep_fpr_firehose(min_fpr, max_fpr, steps, zipf, path, epochs):
    number_of_known_negatives = 20000
    integers = [i for i in range(0,100000)]
    positive_ints = []
    negatives_ints = []
    for i in integers:
        if (random.randint(0,9) % 10) == 0:
            positive_ints += [i]
        else:
            negatives_ints += [i]
    positives = [str(i) for i in positive_ints]
    negatives = [str(i) for i in negatives_ints]
    sizes = []
    fprs = []
    fpr_range = np.geomspace(min_fpr, max_fpr, steps).astype(np.float32)
    model = GRUModel('DeepBloomLib/glove.6B.50d-char.txt', 50,gru_size=1, hidden_size=1, pca_embedding_dim=0, epochs=epochs, batch_size=2048)
    print("Number train, dev", number_of_known_negatives)
    print("Number positives ", len(positives))
    db = train_deep_bloom_filter(positives, negatives[:number_of_known_negatives], model, .01)
    print("Params needed", db.model.model.count_params())
    print("Testing Negatives")
    print("Finished Testing Negatives")
    for i in range(steps-1):
        print(fpr_range[i])
        db.fp_rate = fpr_range[i]
        db.create_fake_bloom_filter(Data(positives, negatives))
        predictions = db.model.predicts(negatives) > db.threshold
        model_false_positive_rate = float(np.sum(predictions*generate_zipf_pmf(4, len(negatives))))
        total_fpr = db.bloom_filter.fp_prob+model_false_positive_rate
        sizes.append(db.bloom_filter.size)
        fprs.append(total_fpr)
        saved_data = pd.DataFrame()
        saved_data["Total FPR"] = fprs
        saved_data["Size of Backup"] = sizes
        saved_data["Bits Per Element"] = (saved_data["Size of Backup"]+512*4*8)/len(positives)
        saved_data["Number of Negatives"] = number_of_known_negatives
        saved_data.to_csv(path, index=False)
    print("Test false positive rate: ", total_fpr)
    print("Bloom filter bits needed", db.bloom_filter.size)

In [ ]:
sweep_fpr_firehose(.5,.001, 10, 1.5, "../Data/learned_bloom_firehose_data_2.csv", positive_strings, negative_strings, 20000, 10)

In [8]:
def sweep_size_sandwiched(min_size, max_size, steps, pmf, path, positive_urls, negative_urls, number_of_known_negatives,epochs):
    sizes = []
    fprs = []
    first_bf_fprs = []
    second_bf_fprs = []
    size_range = np.linspace(min_size, max_size, steps).astype(np.float32)
    model = GRUModel('DeepBloomLib/glove.6B.50d-char.txt', 50, epochs=epochs, batch_size=16384)
    model.load_model("../SavedModels/HALF_NEGATIVES_MODEL_2.h5", positive_urls, negative_urls[:number_of_known_negatives])
    print("Number train, dev", number_of_known_negatives)
    print("Number positives ", len(positive_urls))
    sample = pd.Series(np.random.choice(negative_urls, 500000, True, pmf))
    for i in range(steps):
        db = SandwichedLearnedBloom(model, Data(positives["Domain"], negatives["Domain"][:number_of_known_negatives]), size_range[i], 19208, train_model=False)
        db.create_bloom_filter(Data(positives["Domain"], negatives["Domain"][:number_of_known_negatives]))
        print("Checking Elements")
        predictions =  db.model.predicts(sample)
        print("Checking against threshold")
        predictions = [pred >db.threshold for pred in predictions]
        total_fpr = 0
        for i in range(len(sample)):
            if db.hasInitialFilter and db.bloom_filter1.check(sample[i]) == False:
                continue
            elif predictions[i]:
                total_fpr += 1
            elif db.bloom_filter2.check(sample[i]):
                total_fpr += 1
        total_fpr = total_fpr / len(sample)
        second_bf_fprs.append(db.bloom_filter2.getFPR())
        size = db.bloom_filter2.size + 2369*4*8
        if(db.hasInitialFilter):
            size += db.bloom_filter1.size
            first_bf_fprs.append(db.bloom_filter1.getFPR())
        else:
            first_bf_fprs.append(1)
        size /= len(positive_urls)
        sizes.append(size)
        fprs.append(total_fpr)
        saved_data = pd.DataFrame()
        saved_data["Total FPR"] = fprs
        saved_data["First BF FPR"] = first_bf_fprs
        saved_data["Second BF FPR"] = second_bf_fprs
        saved_data["Bits Per Element"] = sizes
        saved_data["Number of Negatives"] = number_of_known_negatives
        saved_data.to_csv(path, index=False)
    print("Test false positive rate: ", total_fpr)

In [ ]:
sweep_size_sandwiched(4, 14, 11, negatives["PMF"], "../Data/sandwiched_bf_sweep_3.csv", positives["Domain"], negatives["Domain"], 5000000, epochs=50)

In [5]:
def sweep_size_firehose_sandwich(min_size, max_size, steps, path, epochs):
    number_of_known_negatives = 20000
    integers = [i for i in range(0,120000)]
    positive_ints = []
    negatives_ints = []
    for i in integers:
        if (random.randint(0,9) % 10) == 0:
            positive_ints += [i]
        else:
            negatives_ints += [i]
    positives = pd.Series([str(i) for i in positive_ints])
    negatives = pd.Series([str(i) for i in negatives_ints])
    sizes = []
    fprs = []
    first_bf_fprs = []
    second_bf_fprs = []
    size_range = np.linspace(min_size, max_size, steps).astype(np.float32)
    model = GRUModel('DeepBloomLib/glove.6B.50d-char.txt', 50,gru_size=4, hidden_size=4, pca_embedding_dim=0, epochs=epochs, batch_size=2048)
    
    print("Number train, dev", number_of_known_negatives)
    print("Number positives ", len(positives))
    reps = 10
    for i in range(steps):
        db = SandwichedLearnedBloom(model, Data(positives, negatives[:number_of_known_negatives]), size_range[i], 512*4, train_model=True)
        db.create_bloom_filter(Data(positives, negatives[:number_of_known_negatives]))
        predictions = db.model.predicts(negatives) > db.threshold
        total_fpr = 0
        first_bf_fpr = 1
        for j in range(reps):
            db = SandwichedLearnedBloom(model, Data(positives, negatives[:number_of_known_negatives]), size_range[i], 512*4, train_model=True)
            db.create_bloom_filter(Data(positives, negatives[:number_of_known_negatives]))
            model_false_positive_rate = float(np.sum(predictions*generate_zipf_pmf(2, len(negatives))))
            temp_fpr = db.bloom_filter2.fp_prob+model_false_positive_rate
            if(db.hasInitialFilter):
                temp_fpr *= db.bloom_filter1.getFPR()
                first_bf_fpr = min(first_bf_fpr, db.bloom_filter1.getFPR())
            else:
                first_bf_fpr =  min(first_bf_fpr, 1)
            total_fpr += temp_fpr
        total_fpr /=reps
        first_bf_fpr /= reps
        first_bf_fprs.append(first_bf_fpr)
        second_bf_fprs.append(db.bloom_filter2.getFPR())
        sizes.append(size_range[i])
        fprs.append(total_fpr)
        saved_data = pd.DataFrame()
        saved_data["Total FPR"] = fprs
        saved_data["First BF FPR"] = first_bf_fprs
        saved_data["Second BF FPR"] = second_bf_fprs
        saved_data["Bits Per Element"] = sizes
        saved_data["Number of Negatives"] = number_of_known_negatives
        saved_data.to_csv(path, index=False)

In [7]:
sweep_size_firehose_sandwich(8, 14, 4, "../Data/sandwich_firehose_sweep_2_2.csv", 20)

Number train, dev 20000
Number positives  12037
Training model with train, dev, positives 18000 2000 12037
Vectorizing data...
Corpus length 141021
['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
Total chars: 10
[(1, 10), (2, 90), (3, 900), (4, 9000), (5, 18091), (6, 1946)]
max seen length of URL 6
Using maxlen 50
(11, 50)
Epoch 1/20
15/15 - 1s - loss: 0.6767 - accuracy: 0.5856
Epoch 2/20
15/15 - 1s - loss: 0.6712 - accuracy: 0.5993
Epoch 3/20
15/15 - 1s - loss: 0.6643 - accuracy: 0.5993
Epoch 4/20
15/15 - 1s - loss: 0.6468 - accuracy: 0.5993
Epoch 5/20
15/15 - 1s - loss: 0.6158 - accuracy: 0.6727
Epoch 6/20
15/15 - 1s - loss: 0.5564 - accuracy: 0.8367
Epoch 7/20
15/15 - 2s - loss: 0.4665 - accuracy: 0.8818
Epoch 8/20
15/15 - 1s - loss: 0.3592 - accuracy: 0.9203
Epoch 9/20
15/15 - 1s - loss: 0.2833 - accuracy: 0.9310
Epoch 10/20
15/15 - 1s - loss: 0.2481 - accuracy: 0.9321
Epoch 11/20
15/15 - 1s - loss: 0.2380 - accuracy: 0.9321
Epoch 12/20
15/15 - 1s - loss: 0.2325 - accuracy: 0.93